# Blue Brain BioExplorer

![](../bioexplorer_banner.png)

## Simplest script to create a visualization of a cell

In [ ]:
from bioexplorer import BioExplorer, Protein, EnzymeReaction, AnimationParams, \
                        Volume, Vector2, Vector3, Quaternion
be = BioExplorer('localhost:5000')
core = be.core_api()
status = be.reset_scene()

In [ ]:
representation=be.REPRESENTATION_ATOMS_AND_STICKS
atom_radius_multiplier=1.0

# Resources
resource_folder = '../../tests/test_files/'
pdb_folder = resource_folder + 'pdb/'
metabolites_folder = pdb_folder + 'metabolites/'
enzymes_folder = pdb_folder + 'enzymes/'

scene_size = Vector3(200.0, 200.0, 200.0)

# Substrate
substrate = Protein(
    name='Substrate',
    source=metabolites_folder + '5793.pdb',
    occurences=100, animation_params=AnimationParams(1, 0, 0.0, 1, 0.0),
    load_bonds=True, load_hydrogen=True, load_non_polymer_chemicals=True,
)
substrates = Volume(
    name='Substrates',
    shape=be.ASSEMBLY_SHAPE_CUBE, shape_params=scene_size,
    protein=substrate)
status = be.add_volume(volume=substrates, atom_radius_multiplier=atom_radius_multiplier)    

# Product
product = Protein(
    name='Product',
    source=metabolites_folder + '439284.pdb',
    load_bonds=True, load_hydrogen=True, load_non_polymer_chemicals=True,
    occurences=100, animation_params=AnimationParams(2, 0, 0.0, 2, 0.0),
    rotation=Quaternion(0.707, 0.707, 0.0, 0.0),
    chain_ids=[1]
)
products = Volume(
    name='Products',
    shape=be.ASSEMBLY_SHAPE_CUBE, shape_params=scene_size,
    protein=product)
status = be.add_volume(volume=products, atom_radius_multiplier=atom_radius_multiplier)    

# Enzyme
enzyme = Protein(
    name='Enzyme',
    source=enzymes_folder + '1bdg.pdb',
    load_bonds=True, load_hydrogen=True, load_non_polymer_chemicals=True,
    rotation=Quaternion(0.707, 0.707, 0.0, 0.0),
    occurences=100, animation_params=AnimationParams(3, 0, 0.0, 3, 0.0),
    chain_ids=[1]
)
enzymes = Volume(
    name='Enzymes',
    shape=be.ASSEMBLY_SHAPE_CUBE, shape_params=scene_size,
    protein=enzyme)
status = be.add_volume(volume=enzymes, atom_radius_multiplier=atom_radius_multiplier)

enzyme_reaction = EnzymeReaction(
    assembly_name='Hexokinase',
    name='Hexokinase',
    enzyme=enzyme,
    substrate=substrate, substrate_instance=0,
    product=product
)

be.add_enzyme_reaction(enzyme_reaction, progress=0.5)

In [140]:
instance_id = 0
# be.set_focus_on(
#     model_id=0, instance_id=instance_id, distance=10.0)
be.set_enzyme_reaction_progress(
    enzyme_reaction, instance_id=instance_id,
    progress=1.0)
# be.core_api().set_renderer()

{'contents': '', 'status': True}

## Rendering settings

In [ ]:
status = be.core_api().set_renderer(
    background_color=[96 / 255, 125 / 255, 139 / 255],
    current='bio_explorer',
    samples_per_pixel=1, subsampling=4, max_accum_frames=1000)
params = status = be.core_api().BioExplorerRendererParams()
params.gi_samples = 1
params.gi_weight = 0.3
params.gi_distance = 5
params.shadows = 0.8
params.soft_shadows = 1
params.fog_start = 1500
params.fog_thickness = 1500
params.max_bounces = 1
status = be.core_api().set_renderer_params(params)

## Materials

In [ ]:
status = be.set_protein_color_scheme(
    assembly_name='Products', name='Product',
    color_scheme=be.COLOR_SCHEME_ATOMS,
    palette_name='Greens', palette_size=5)

status = be.set_protein_color_scheme(
    assembly_name='Substrates', name='Substrate',
    color_scheme=be.COLOR_SCHEME_ATOMS,
    palette_name='Blues', palette_size=5)

status = be.set_protein_color_scheme(
    assembly_name='Enzymes', name='Enzyme',
    color_scheme=be.COLOR_SCHEME_ATOMS,
    palette_name='Set1', palette_size=5)

## Movie

In [ ]:
from bioexplorer import MovieMaker
mm = MovieMaker(be)

if True:
    import time
    for i in range(30):
        be.set_enzyme_reaction_progress(
            enzyme_reaction, instance_id=instance_id,
            progress=i / 30.0)
        # be.core_api().set_renderer()
        # time.sleep(0.2)
        mm.create_snapshot(
            renderer='bio_explorer',
            path='/tmp', base_name='%05d' % i,
            size=[512,512], samples_per_pixel=64)
